In [840]:
import pandas as pd
df_train = pd.read_csv('clean_train.csv')
y = df_train['Survived']
df_train = df_train.drop(columns=['Survived'])
df_test = pd.read_csv('clean_test.csv')
df_train.head()

,Pclass,Age,SibSp,Parch,Fare,TicketDigits,nanAge,nanFare,nanEmbarked,nanCabinLetter,...,FirstName_Yousseff,FirstName_of,CabinLetter_A,CabinLetter_B,CabinLetter_C,CabinLetter_D,CabinLetter_E,CabinLetter_F,CabinLetter_G,CabinLetter_T
0,3,22.0,1,0,7.2500,21171,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1,38.0,1,0,71.2833,17599,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,3,26.0,0,0,7.9250,3101282,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1,35.0,1,0,53.1000,113803,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,3,35.0,0,0,8.0500,373450,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [841]:
def print_moderate_correlations(df, col_to_correlate):
    cols = df[df.columns].corr().columns
    if df[col_to_correlate].dtype.name == 'category':
        df[col_to_correlate] = df[col_to_correlate].cat.codes
    corrs = df[df.columns].corr()[col_to_correlate]
    for col, corr in zip(cols, corrs):
        if abs(corr) >= 0.1 and col != col_to_correlate:
            print(col, ': ', corr)

In [842]:
df_train['Survived'] = y
print_moderate_correlations(df_train, 'Survived')
df_train = df_train.drop(columns=['Survived'])

Pclass :  -0.33848103596101586
Parch :  0.08162940708348222
Fare :  0.2573065223849618
TicketDigits :  -0.115156746957067
isAlone :  -0.2033670856998897
is_french :  0.07364228638655194
is_religious :  -0.06498765565557524
is_nobility :  0.0505614454248609
Sex_female :  0.5433513806577526
Sex_male :  -0.5433513806577526
Embarked_C :  0.16824043121823343
Embarked_S :  -0.14968272327068607
LastName_Asplund :  0.050561445424861816
LastName_Baclini :  0.08508270816321468
LastName_Beane :  0.060094847378357096
LastName_Becker :  0.060094847378357685
LastName_Beckwith :  0.06009484737835756
LastName_Bishop :  0.0600948473783575
LastName_Brown :  0.05056144542486166
LastName_Caldwell :  0.060094847378357845
LastName_Chambers :  0.0600948473783568
LastName_Coutts :  0.06009484737835739
LastName_Daly :  0.06009484737835714
LastName_Dick :  0.060094847378356846
LastName_Doling :  0.06009484737835781
LastName_Duff Gordon :  0.06009484737835707
LastName_Ford :  -0.053002342790198075
LastName_Fraue

In [843]:
def remove_weak_correlations(df1: pd.DataFrame, df2, y, weak_threshold: float = 0.1) -> pd.DataFrame:
    """
    Removes weak correlations
    :param df: pandas DataFrame to remove columns from.
    :param col_to_correlate: String column name to check correlation with
    :param weak_threshold: float number that counts as an absolute weak threshold
    :return: pandas DataFrame without the columns weakly correlated to target
    """
    col_to_correlate = 'PREDICTOR_TO_CHECK_WEAK_CORRS'
    df1[col_to_correlate] = y
    cols = df1[df1.columns].corr().columns
    corrs = df1[df1.columns].corr()[col_to_correlate]
    strongly_correlated = list()
    for col, corr in zip(cols, corrs):
        if abs(corr) >= weak_threshold and col != col_to_correlate:
            strongly_correlated.append(col)
    df1 = df1.drop(columns=[col_to_correlate])
    for col in df1.columns:
        if col not in strongly_correlated:
            df1.drop(columns=[col], inplace=True)
            df2.drop(columns=[col], inplace=True)
    return df1, df2

In [844]:
def get_rid_feature(df, df2, feat):
    for column in df_train:
        if column.startswith(feat):
            df_train.drop(columns=[column], inplace=True)
            df_test.drop(columns=[column], inplace=True)
    return df_train, df_test

In [845]:
df_train, df_test = remove_weak_correlations(df_train, df_test, y)

In [846]:
# df_train, df_test = get_rid_feature(df_train, df_test, 'Parch')
# df_train, df_test = get_rid_feature(df_train, df_test, 'LastName')
# df_train, df_test = get_rid_feature(df_train, df_test, 'FirstName')
# df_train, df_test = get_rid_feature(df_train, df_test, 'Sex_male')
# df_train, df_test = get_rid_feature(df_train, df_test, 'SibSp')
# df_train, df_test = get_rid_feature(df_train, df_test, 'Fare')
# df_train, df_test = get_rid_feature(df_train, df_test, 'Age')

In [847]:
df_train.head()

,Pclass,Fare,TicketDigits,isAlone,Sex_female,Sex_male,Embarked_C,Embarked_S,Title_Mr,Title_Mrs,Title_Ms,FirstName_Anna,FirstName_Elizabeth,FirstName_Margaret,CabinLetter_B,CabinLetter_C,CabinLetter_D,CabinLetter_E,CabinLetter_F
0,3,7.2500,21171,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1
1,1,71.2833,17599,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0
2,3,7.9250,3101282,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1
3,1,53.1000,113803,0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,0
4,3,8.0500,373450,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1


In [ ]:
def print_moderate_correlations(df, col_to_correlate):
    cols = df[df.columns].corr().columns
    if df[col_to_correlate].dtype.name == 'category':
        df[col_to_correlate] = df[col_to_correlate].cat.codes
    corrs = df[df.columns].corr()[col_to_correlate]
    for col, corr in zip(cols, corrs):
        if abs(corr) > 0.1 and col != col_to_correlate:
            print(col, ': ', corr)

In [ ]:
# df_train, df_test = get_rid_feature(df_train, df_test, 'CabinLetter')

In [ ]:
df_train.head()

,Pclass,Fare,TicketDigits,isAlone,Sex_female,Sex_male,Embarked_C,Embarked_S,Title_Mr,Title_Mrs,Title_Ms,FirstName_Anna,FirstName_Elizabeth,FirstName_Margaret,CabinLetter_B,CabinLetter_C,CabinLetter_D,CabinLetter_E,CabinLetter_F
0,3,7.2500,21171,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1
1,1,71.2833,17599,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0
2,3,7.9250,3101282,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1
3,1,53.1000,113803,0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,0
4,3,8.0500,373450,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1


In [ ]:
from sklearn.model_selection import train_test_split

x = df_train.values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.10, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
import numpy as np
X = np.concatenate((x_train, x_test), axis=0)
Y = np.concatenate((y_train, y_test), axis=0)

In [ ]:
from sklearn.model_selection import cross_val_score

def evaluate_classifier(clf, x, y):
    scores = cross_val_score(gb_clf, x, y, cv=10)
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2)) 

In [ ]:
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier

gb_clf = GradientBoostingClassifier(criterion='friedman_mse', init=None,
             learning_rate=0.05, max_depth=3,
             max_features='sqrt', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=15, min_samples_split=10,
             min_weight_fraction_leaf=0.0, n_estimators=12000,
             presort='auto', subsample=1.0, verbose=0,
             warm_start=False)

evaluate_classifier(gb_clf, X, Y)

Accuracy: 0.79 (+/- 0.04)


In [ ]:
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier

gb2_clf = GradientBoostingClassifier(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   random_state = 5)

evaluate_classifier(gb2_clf, X, Y)

Accuracy: 0.79 (+/- 0.05)


In [ ]:
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression(penalty='l2', tol=0.0001, solver='lbfgs',
                           C=1, fit_intercept=True, intercept_scaling=1.0, 
                           class_weight=None, random_state=None)

evaluate_classifier(lr_clf, X, Y)

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp_clf = MLPClassifier(max_iter=1000, hidden_layer_sizes=(100,10), learning_rate='adaptive')

evaluate_classifier(mlp_clf, x_train, y_train)

In [ ]:
df_sub = pd.read_csv('gender_submission.csv')
df_sub.head()

In [ ]:
sub_model = gb_clf

In [ ]:
gb_clf.fit(X, Y)

In [ ]:
from sklearn.model_selection import cross_val_predict
df_sub['Survived'] = gb_clf.predict(df_test.values)
df_sub.to_csv('submission.csv', index=False)